In [ ]:
"""
[REDACTED]/.py

This script fetches the latest Windows OS build lifecycle information from the public
endoflife.date API and generates a structured Word document (.docx) for client-facing
communication. It identifies the most recent supported and unsupported Windows builds,
formats dates for clarity, and builds a narrative with customizable sections.

Use Case:
---------
Primarily used by MSPs or IT operations teams to notify clients about upcoming
end-of-life milestones, required build upgrades, and maintenance schedules.

Key Features:
-------------
- Queries endoflife.date API for Windows lifecycle metadata
- Filters editions such as (E), (W), and (E)(W) based on regex matching
- Identifies the newest build and next version reaching end-of-support
- Formats a professional message with customizable text blocks
- Outputs a .docx file suitable for distribution to clients

Output:
-------
A Microsoft Word document saved to the local `export_dir` with a timestamped narrative
on lifecycle status and upgrade timelines.
"""


In [ ]:
import pandas as pd
import re
import os
import datetime as dt
import requests
from docx import *

"""
(E) 	Enterprise, Education and IoT Enterprise editions
(W) 	Home, Pro, Pro Education and Pro for Workstations editions
LTS 	Long-Term Servicing Channel
"""
export_dir = "d:/exports"


# Variables

class WindowsBuildResults:
    def __init__(self):
        self.__df = self.__get_endoflife__()

    @staticmethod
    def __get_endoflife__():
        resp = requests.get("[REDACTED]/.json")
        df = pd.read_json(resp.text)
        df[["releaseDate", "support", "eol"]] = df[["releaseDate", "support", "eol"]].apply(pd.to_datetime)
        return df

    def eol_dict(self):
        win_os_nums = []
        min_build_list = []
        max_build_list = []
        df_dict = {
            "eol": self.__df[self.__df["support"] < dt.datetime.now()].head(10).sort_values(by="support",
                                                                                            ascending=False),
            "newest": self.__df[self.__df["support"] > dt.datetime.now()].head(10).sort_values(by="releaseDate",
                                                                                               ascending=False)
        }

        for k, v in df_dict.items():
            win_os_nums = []
            for index, row in v.iterrows():
                result = re.match(r"(\d{2}),\sversion\s(\w{4})\s?([\(\)EWLTSIoT]+)\s?",
                                  row["cycle"])
                # result = re.match(r"(\d{2}),\sversion\s(\w{4})(\s?\(E\)\s?|\s?\(W\)\s?|\s?\(E\)\(W\)\s?|\s?LTS\s?|\s?IoT.*|\s?)?",
                #                   row["cycle"])
                if result:
                    win_num = result.group(1)
                    win_edi = (result.group(3)).strip()
                    if win_num not in win_os_nums:
                        if (win_edi == "(W)") | (win_edi == "(E)(W)"):
                            print(win_edi)
                            win_os_nums.append(result.group(1))
                            if k == "newest":
                                max_build_list.append({
                                    "win_num": win_num,
                                    "release_name": result.group(2),
                                    "release_date": row["releaseDate"],
                                    "end_of_support_date": row["support"],
                                    "edition": win_edi,
                                    "is_lts": row["lts"],
                                    "link": row["link"]

                                })
                            elif k == "eol":
                                min_build_list.append({
                                    "win_num": win_num,
                                    "release_name": result.group(2),
                                    "release_date": row["releaseDate"],
                                    "end_of_support_date": row["support"],
                                    "edition": win_edi,
                                    "is_lts": row["lts"],
                                    "link": row["link"]

                                })

        return {
            "win_os_nums": win_os_nums,
            "newest":{
            "build_list": max_build_list,
            "df": df_dict["newest"]
            },
            "eol":{
            "build_list": min_build_list,
            "df": df_dict["eol"]
            }
        }

In [ ]:
windows_builds = WindowsBuildResults()

In [ ]:
results = windows_builds.eol_dict()

In [ ]:
results["eol"]["df"]

In [ ]:
results["newest"]["build_list"]

In [ ]:
results["eol"]["build_list"]

# Format

In [ ]:
def add_suffix_to_day(day):
    day = int(day)
    day_endings = {
        1: 'st',
        2: 'nd',
        3: 'rd',
        21: 'st',
        22: 'nd',
        23: 'rd',
        31: 'st'
    }

    suffix = day_endings.get(day, 'th')
    num = re.sub(r"0\d","",str(day))

    return num + suffix + ","

In [ ]:
windows_os = results["win_os_nums"]

In [ ]:
if len(windows_os) == 1:
    os_desc = f"Windows {windows_os[0]}"
    s = ""
elif len(windows_os) == 2:
    os_desc = f"Windows {windows_os[0]} and {windows_os[1]}"
    s = "s"
else:
    exit(1)

title = f"Microsoft {os_desc} Build Updates"
newest_version = results["newest"]["build_list"][0]["release_name"]
newest_version_date = " ".join([
    (results["newest"]["build_list"][0]["release_date"]).month_name(),
    add_suffix_to_day(results["newest"]["build_list"][0]["release_date"].day),
    str((results["newest"]["build_list"][0]["release_date"]).year)
])



min_end_of_life_version = results["eol"]["build_list"][0]["release_name"]
min_end_of_life_version_date = " ".join([
    (results["eol"]["build_list"][0]["end_of_support_date"]).month_name(),
    add_suffix_to_day(results["eol"]["build_list"][0]["end_of_support_date"].day),
     str((results["eol"]["build_list"][0]["end_of_support_date"]).year)
    ])

when_we_start_updating_time = "11 PM"
when_we_start_updating_date = " ".join([
    dt.datetime.now().strftime("%B"),

    add_suffix_to_day((dt.datetime.now() + dt.timedelta(7)).strftime("%d")),

    dt.datetime.now().strftime("%Y")
])

In [ ]:

# Title
def message_greeting(doc):
    p = doc.add_paragraph(f"""
Hi there,

As you may know, Microsoft feature updates for {os_desc} build{s} are released annually, in the second half of the calendar year, {newest_version_date} to be specific, to the General Availability Channel. Each release will be serviced with monthly quality updates for 18 or 30 months from the date of the release, depending on the lifecycle policy. After which period, the version would be cycled out.  The current release{s} for {os_desc} is build {newest_version}.
""")

    p.add_run(
        f'\n\nThe latest build to reach end of life is {os_desc} build version {min_end_of_life_version}, which will reach end of life {min_end_of_life_version_date}.\n').bold = True

    return doc


what_should_i_look_out_for_list = [
    f"Make sure all {os_desc} devices are online during maintenance windows, including remote laptop users.",
    f"For sites with limited internet bandwidth, there could be diminished network performance during the upgrade due to a large number of devices simultaneously downloading the new build",
    f"If any computers in your network experience problems loading specific software after the build update, we recommend that you reach out to the application vendor to confirm compatibility with {os_desc} build {min_end_of_life_version}."
]



def what_this_means(doc):
    p = doc.add_paragraph()

    p.add_run("\nWhat This Means:").bold = True

    doc.add_paragraph(f"""
No security or feature update patches will be available for this version or below. This includes patching deployed via Example Co. patch automation, as Microsoft will discontinue mainstream support for devices running this build version.
""")
    return doc


def what_happens_next(doc):
    p = doc.add_paragraph()

    p.add_run("\nWhat Happens Next:").bold = True

    doc.add_paragraph(f"""
Example Co. will be updating all {os_desc} machines at or below build {min_end_of_life_version} starting this {when_we_start_updating_date}, at {when_we_start_updating_time} EST. This update will only run during the maintenance window and will only be in effect for clients with active NOC Patching Service. Devices must be online during the maintenance window in order to receive this update.

If you do not currently subscribe to Example Co. NOC Patching Service, we recommend ensuring all your devices are at or above build {min_end_of_life_version}. Please feel free to reach out if we can be of assistance.
""")

    return doc


def what_should_i_look_out_for(doc):
    p = doc.add_paragraph()
    p.add_run(f'\nWhat Should I Look Out For?\n').bold = True
    for item in what_should_i_look_out_for_list:
        doc.add_paragraph(f'{item}\n', style='List Bullet')

    return doc


def what_if_problems(doc):
    p = doc.add_paragraph()

    p.add_run("\nWhat If Problems Should Arise?").bold = True

    doc.add_paragraph(f"""
Since {os_desc} previous build versions share a common core operating system and an identical set of system files we expect minimal impact. If any issues are experienced, we have a script created to rollback this update.

Our team is taking all precautions to minimize any business disruptions while balancing security impact. If you would like to be excluded from this update or have software compatibility issues, please contact support@example.co.
""")

    return doc


def if_you_have_more_questions(doc):
    p = doc.add_paragraph()

    p.add_run("\nNeed something else?").bold = True

    doc.add_paragraph(f"""
If you have any questions or need help from our team, please contact us at support@example.co or call at 410.560.5601


Thank you,
The Example Co. Team

""").italics = True

    return doc


def for_more_information(doc):
    p = doc.add_paragraph()

    p.add_run("\nFor More Information:").bold = True

    for i in windows_os:
        try:
            microsoft_lifecycle_policy = f"https://docs.microsoft.com/en-us/lifecycle/products/windows-{i}-home-and-pro"
            resp = requests.get(microsoft_lifecycle_policy)
            if resp.status_code == 200:
                doc.add_paragraph(f'Microsoft Statement on {i} Lifecycle:\n{microsoft_lifecycle_policy}\n')
            else:
                print(f"Failed to get {microsoft_lifecycle_policy}")
        except Exception as e:
            print(e)
            doc.add_paragraph(f'\n\n')

    for i in windows_os:
        microsoft_eol_policy = f"https://docs.microsoft.com/en-us/lifecycle/announcements/windows-10-{min_end_of_life_version}-end-of-servicing"
        try:
            resp = requests.get(microsoft_eol_policy)
            if resp.status_code == 200:
                doc.add_paragraph(f'Microsoft Statement on {i} EOL:\n{microsoft_eol_policy}\n')
        except Exception as e:
            print(e)

    return doc


document = Document()
document = message_greeting(document)
document = what_this_means(document)
document = what_happens_next(document)
document.add_page_break()
document = what_should_i_look_out_for(document)
document = what_if_problems(document)
document = if_you_have_more_questions(document)
document.add_page_break()
document = for_more_information(document)

document.save(f"{export_dir}/operating_system_lifecycle.docx")
